# Работа с Excel

Материалы:
* Макрушин С.В. Лекция 7: Работа с Excel
* https://docs.xlwings.org/en/stable/quickstart.html
* https://nbviewer.jupyter.org/github/pybokeh/jupyter_notebooks/blob/master/xlwings/Excel_Formatting.ipynb#search_text


## Задачи для совместного разбора

In [1]:
import numpy as np
import xlwings as xw
import pandas as pd
from pprint import pprint as pp


1. На листе "Рецептура" файла `себестоимостьА_в1.xlsx` для области "Пшеничный хлеб" рассчитать себестоимость всех видов продукции.

In [3]:
wb = xw.Book('./data/data/себестоимостьА_в1.xlsx')

In [4]:
sheet = wb.sheets['Рецептура']

In [10]:
consumption = sheet.range('G7:O10').options(np.array).value
consumption

array([[0.2  , 0.4  , 0.02 , 0.02 , 0.01 , 0.005, 0.1  ,   nan,   nan],
       [0.07 , 0.25 , 0.01 , 0.06 , 0.015,   nan, 0.06 , 0.05 ,   nan],
       [0.12 , 0.22 , 0.005,   nan, 0.009, 0.005,   nan, 0.05 , 0.03 ],
       [0.12 , 0.23 , 0.01 ,   nan, 0.015,   nan,   nan,   nan, 0.05 ]])

In [9]:
unit_price = sheet.range('G14:O14').options(np.array).value
unit_price

array([  7.,  10., 184.,  19.,   7., 290., 105.,  83., 260.])

In [14]:
res = np.nan_to_num(consumption * unit_price).sum(axis=1)
res

array([21.48 , 16.525, 17.423, 18.085])

2. Результаты расчетов 1.1 сохранить в отдельном столбце области "Пшеничный хлеб"

In [17]:
sheet.range('T7:T10').options(transpose=True).value = res

3. Приблизить форматирование столбца, добавленного в задаче 2 к оформлению всей области.

In [20]:
sheet.range('T6').value = 'Себестоимость'
sheet.range('T4:T6').api.Merge()
sheet.range('T4:T6').color = (255, 0, 255)

4. Выполнить 3 с помощью "протягиваемых" формул.

In [22]:
from xlwings.constants import AutoFillType

In [23]:
sheet.range('V7').formula = '=SUMPRODUCT(G7:O7, $G$14:$O$14)'
sheet.range('V7').api.AutoFill(sheet.range('V7:V10').api,
                               AutoFillType.xlFillDefault)

True

## Лабораторная работа 7

1. Загрузите данные из файлов `reviews_sample.csv` (__ЛР2__) и `recipes_sample_with_tags_ingredients.csv` (__ЛР5__) в виде `pd.DataFrame`. Обратите внимание на корректное считывание столбца(ов) с индексами. Оставьте в таблице с рецептами следующие столбцы: `id`, `name`, `minutes`, `submitted`, `description`, `n_ingredients`

In [14]:
reviews_sample = pd.read_csv("./data/data/reviews_sample.csv", index_col=0, parse_dates=["date"])
recipes_sample_with_tags_ingredients = \
    pd.read_csv("./data/data/recipes_sample_with_tags_ingredients.csv", parse_dates=["submitted"])\
        .drop(columns=["contributor_id", "n_steps", "tags", "n_tags", "recipe_id", "ingredients"])

In [15]:
reviews_sample.head()

,user_id,recipe_id,date,rating,review
370476,21752,57993,2003-05-01,5,Last week whole sides of frozen salmon fillet ...
624300,431813,142201,2007-09-16,5,So simple and so tasty! I used a yellow capsi...
187037,400708,252013,2008-01-10,4,"Very nice breakfast HH, easy to make and yummy..."
706134,2001852463,404716,2017-12-11,5,These are a favorite for the holidays and so e...
312179,95810,129396,2008-03-14,5,Excellent soup! The tomato flavor is just gre...


In [16]:
recipes_sample_with_tags_ingredients.head()

,name,id,minutes,submitted,description,n_ingredients
0,george s at the cove black bean soup,44123,90,2002-10-25,an original recipe created by chef scott meska...,18
1,healthy for them yogurt popsicles,67664,10,2003-07-26,my children and their friends ask for my homem...,3
2,i can t believe it s spinach,38798,30,2002-08-29,"these were so go, it surprised even me.",8
3,italian gut busters,35173,45,2002-07-27,my sister-in-law made these for us at a family...,9
4,love is in the air beef fondue sauces,84797,25,2004-02-23,i think a fondue is a very romantic casual din...,12


2. Случайным образом выберите 5% строк из каждой таблицы и сохраните две таблицы на разные листы в один файл `recipes.xlsx`. Дайте листам названия "Рецепты" и "Отзывы", соответствующие содержанию таблиц. 

In [33]:
from openpyxl import load_workbook

book = load_workbook("./data/data/recipes.xlsx")
writer = pd.ExcelWriter("./data/data/recipes.xlsx", engine = 'openpyxl')
reviews_sample.to_excel(writer, sheet_name = 'Отзывы')
recipes_sample_with_tags_ingredients.to_excel(writer, sheet_name = 'Рецепты')
writer.save()
writer.close()

IllegalCharacterError: 

In [30]:
reviews_sample.to_excel("./data/data/recipes.xlsx", sheet_name="Отзывы", engine='xlsxwriter')


In [ ]:
recipes_sample_with_tags_ingredients.to_excel("./data/data/recipes.xlsx", sheet_name="Рецепты", engine='xlsxwriter')

3. Используя `xlwings`, добавьте на лист `Рецепты` столбец `seconds_assign`, показывающий время выполнения рецепта в секундах. Выполните задание при помощи присваивания массива значений диапазону ячеек.

In [26]:
wb = xw.Book('./data/data/recipes.xlsx')
# wb_recipes_sheet = wb.sheets['Рецепты']
# wb_reviews_sheet = wb.sheets['Отзывы']

com_error: (-2147352567, 'Ошибка.', (0, None, None, None, 0, -2147352565), None)

In [27]:
wb.close()

com_error: (-2147023174, 'Сервер RPC недоступен.', None, None)

4. Используя `xlwings`, добавьте на лист `Рецепты` столбец `seconds_formula`, показывающий время выполнения рецепта в секундах. Выполните задание при помощи формул Excel.

5. Добавьте на лист `Рецепты`  столбец `n_reviews`, содержащий кол-во отзывов для этого рецепта. Выполните задание при помощи формул Excel.

6. Сделайте названия всех добавленных столбцов полужирными и выровняйте по центру ячейки.

7. Раскрасьте ячейки столбца `minutes` в соответствии со следующим правилом: если рецепт выполняется быстрее 5 минут, то цвет - зеленый; от 5 до 10 минут - жёлтый; и больше 10 - красный.

8. Напишите функцию `validate()`, которая проверяет соответствие всех строк из листа `Отзывы` следующим правилам:
    * Рейтинг - это число от 0 до 5 включительно
    * Соответствующий рецепт имеется на листе `Рецепты`
    
В случае несоответствия этим правилам, выделите строку красным цветом